# LapLab / DataFrames <br>
- Read original CSV source file into dataframes <br>
- Clean data & maintain data features based on Schema <br>
- Modify dataframes & create MySQL source files as CSVs <br>

### 1. Read original CSV file

In [341]:
import pandas as pd

In [342]:
df_orig = pd.read_csv("../laplab_refer/sales_data.csv")
df_orig.head()

,Order_ID,Branch,Order_Date,Order_Priority,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,...,GPU,OS,OS_Version,Weight,Price,Quantity,Discount,Total_Price,Profit,Ship_Duration
0,0,Hamedan,1396-10-26,H,Dell,Vostro 3568,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,...,AMD Radeon R5 M420,Windows,10,2.18kg,3450000,1,0,3450000.0,250000,3
1,1,Kermanshah,1394-04-21,H,Lenovo,Yoga 720-13IKB,2 in 1 Convertible,"13.3""",IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,...,Intel HD Graphics 620,Windows,10,1.3kg,3400000,1,0,3400000.0,380000,2
2,2,Rasht,1402-05-24,M,Lenovo,IdeaPad Y700-15ISK,Notebook,"15.6""",IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,...,Nvidia GeForce GTX 960M,Windows,10,2.6kg,44900000,1,0,44900000.0,5630000,5
3,3,Kermanshah,1397-06-17,H,Dell,Inspiron 5370,Ultrabook,"13.3""",IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,...,AMD Radeon 530,Windows,10,1.4kg,12550000,1,0,12550000.0,1130000,3
4,4,Karaj,1392-09-30,L,Acer,Aspire E5-576G,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,...,Nvidia GeForce 940MX,Windows,10,2.23kg,2050000,1,0,2050000.0,180000,8


In [343]:
# just first 5000 rows
df = df_orig.copy()

### 2. Clean data & Maintain Features

In [344]:
# Scree_Size :: Remove " and convert it to float

df["Screen_Size"] = df["Screen_Size"].str.replace('"', "").astype(float)

In [345]:
# Screen :: Split Screen into Screen_Type and Screen_resolution

df[["Screen_Type", "Screen_resolution"]] = df["Screen"].str.extract(r"(.*?)(\d+x\d+)$")
df.drop(columns=["Screen"], axis=1, inplace=True)

In [346]:
# CPU :: Split into 'CPU_Brand', 'CPU_Model' & 'CPU_GHZ'

pattern_cpu = r"^(\w+)\s(.+?)\s(\d+\.\d+)GHz"
df[["CPU_Brand", "CPU_Model", "CPU_GHZ"]] = df["CPU"].str.extract(pattern_cpu)
df["CPU_GHZ"] = df["CPU_GHZ"].astype(float)
df.drop(columns=["CPU"], axis=1, inplace=True)

In [347]:
# RAM :: Remove GB and convert it to int

df["RAM"] = df["RAM"].str.replace("GB", "").astype(int)

In [348]:
# # Storage :: Split into 'Storage_HDD', 'Storage_SSD', 'Storage_Hybrid', 'Storage_Flash' & 'Storage_Total'

# def extract_storage(value, storage_type):
#     pattern_storage = rf"(\d+)(GB|TB) {storage_type}"
#     matches = pd.Series(value).str.findall(pattern_storage)
#     total_storage = 0

#     for match in matches.iloc[0]:
#         storage, unit = match
#         storage = int(storage)
#         if unit == "TB":
#             storage *= 1024  # Convert TB to GB
#         total_storage += storage

#     return total_storage


# df["Storage_SSD"] = df["Storage"].apply(extract_storage, storage_type="SSD")
# df["Storage_HDD"] = df["Storage"].apply(extract_storage, storage_type="HDD")
# df["Storage_Hybrid"] = df["Storage"].apply(extract_storage, storage_type="Hybrid")
# df["Storage_Flash"] = df["Storage"].apply(extract_storage, storage_type="Flash")
# df["Storage_Total"] = (df["Storage_SSD"] + df["Storage_HDD"] + df["Storage_Hybrid"] + df["Storage_Flash"])
# df.drop(columns="Storage", axis=1, inplace=True)

In [349]:
# Storage :: Split into 'Storage_HDD', 'Storage_SSD', 'Storage_Hybrid', 'Storage_Flash' & 'Storage_Total'
# Optimized Version

def extract_all_storage(df, storage_type):
    pattern_storage = rf"(\d+)(GB|TB) {storage_type}"
    extracted = df['Storage'].str.extractall(pattern_storage)
    extracted[0] = extracted[0].astype(int)
    extracted.loc[extracted[1] == 'TB', 0] *= 1024
    total_storage = extracted.groupby(level=0)[0].sum()
    
    return df.index.map(total_storage).fillna(0).astype(int)

df["Storage_SSD"] = extract_all_storage(df, "SSD")
df["Storage_HDD"] = extract_all_storage(df, "HDD")
df["Storage_Hybrid"] = extract_all_storage(df, "Hybrid")
df["Storage_Flash"] = extract_all_storage(df, "Flash")
df["Storage_Total"] = df[["Storage_SSD", "Storage_HDD", "Storage_Hybrid", "Storage_Flash"]].sum(axis=1)

df.drop(columns="Storage", inplace=True)

In [350]:
# GPU :: Split into 'GPU_Brand' & 'GPU_Model'

pattern = r"^(\w+)\s(.+)$"
df[["GPU_Brand", "GPU_Model"]] = df["GPU"].str.extract(pattern)
df.drop(columns="GPU", axis=1, inplace=True)

In [351]:
# OS :: Clean OS names

def clean_os_name(name):
    if "mac" in name.lower():
        return "macOS"
    elif "chrome" in name.lower():
        return "ChromeOS"

    return name

df["OS"] = df["OS"].apply(clean_os_name)

In [352]:
# Weight :: Remove kg and convert it to float

df["Weight"] = df["Weight"].str.extract(r"(\d+\.\d+|\d+)").astype(float)

In [353]:
# Total Price :: Change to Int

df["Total_Price"] = df["Total_Price"].astype(int)

In [354]:
# Rating :: Create extra column for laptop ratings

df["Rating"] = {}

In [355]:
# Store :: Create extra column for store name selling laptops

df["Store"] = {}

In [356]:
df.head()

,Order_ID,Branch,Order_Date,Order_Priority,Manufacturer,Model_Name,Category,Screen_Size,RAM,OS,...,CPU_GHZ,Storage_SSD,Storage_HDD,Storage_Hybrid,Storage_Flash,Storage_Total,GPU_Brand,GPU_Model,Rating,Store
0,0,Hamedan,1396-10-26,H,Dell,Vostro 3568,Notebook,15.6,8,Windows,...,2.5,256,0,0,0,256,AMD,Radeon R5 M420,NaN,NaN
1,1,Kermanshah,1394-04-21,H,Lenovo,Yoga 720-13IKB,2 in 1 Convertible,13.3,8,Windows,...,2.5,256,0,0,0,256,Intel,HD Graphics 620,NaN,NaN
2,2,Rasht,1402-05-24,M,Lenovo,IdeaPad Y700-15ISK,Notebook,15.6,8,Windows,...,2.6,0,1024,0,0,1024,Nvidia,GeForce GTX 960M,NaN,NaN
3,3,Kermanshah,1397-06-17,H,Dell,Inspiron 5370,Ultrabook,13.3,8,Windows,...,1.8,256,0,0,0,256,AMD,Radeon 530,NaN,NaN
4,4,Karaj,1392-09-30,L,Acer,Aspire E5-576G,Notebook,15.6,4,Windows,...,2.5,256,0,0,0,256,Nvidia,GeForce 940MX,NaN,NaN


### 3. Modify dataframes & output CSVs

In [357]:
# Manufacturer :: Create Schematic dataframe & CSV

Manufacturer_cols = ["Manufacturer"]

df_Manufacturer = (
    df[Manufacturer_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_Manufacturer["index"] += 1

df_Manufacturer.rename(columns={"index": "Manufacturer_ID"}, inplace=True)

df_Manufacturer.to_csv("../laplab_refer/csv_4_sql/Manufacturer.csv", index=False)

df_Manufacturer.head()

,Manufacturer_ID,Manufacturer
0,1,Dell
1,2,Lenovo
2,3,Acer
3,4,HP
4,5,Fujitsu


In [358]:
# Create function to add Manufacturer_ID to the given dataframe and drop Manufacturer column


def add_Manufacturer_ID(df):
    df = df.merge(df_Manufacturer, on="Manufacturer", how="left")
    df.drop(columns=["Manufacturer"], axis=1, inplace=True)

    return df

In [359]:
# OperatingSystem :: Create Schematic dataframe & CSV

OperatingSystem_cols = ["OS", "OS_Version"]

df_OperatingSystem = (
    df[OperatingSystem_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_OperatingSystem["index"] += 1

df_OperatingSystem.rename(columns={"index": "OperatingSystem_ID"}, inplace=True)

df_OperatingSystem.to_csv("../laplab_refer/csv_4_sql/OperatingSystem.csv", index=False)

df_OperatingSystem.head()

,OperatingSystem_ID,OS,OS_Version
0,1,Windows,10
1,2,ChromeOS,NaN
2,3,Linux,NaN
3,4,Windows,7
4,5,macOS,X


In [360]:
# Create function to add OperatingSystem_ID to the given dataframe and drop OS & OS_Version columns

def add_OperatingSystem_ID(df):
    df = df.merge(df_OperatingSystem, on=["OS", "OS_Version"], how="left")
    df.drop(columns=["OS", "OS_Version"], axis=1, inplace=True)

    return df

In [361]:
# CPU :: Create Schematic dataframe & CSV

CPU_cols = ["CPU_Brand", "CPU_Model", "CPU_GHZ"]

df_CPU = (
    df[CPU_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_CPU["index"] += 1

df_CPU.rename(columns={"index": "CPU_ID"}, inplace=True)

df_CPU.to_csv("../laplab_refer/csv_4_sql/CPU.csv", index=False)

df_CPU.head()

,CPU_ID,CPU_Brand,CPU_Model,CPU_GHZ
0,1,Intel,Core i5 7200U,2.5
1,2,Intel,Core i7 6700HQ,2.6
2,3,Intel,Core i7 8550U,1.8
3,4,AMD,FX 8800P,2.1
4,5,Intel,Core i7 7500U,2.7


In [362]:
# Create function to add CPU_ID to the given dataframe and drop CPU_Brand, CPU_Model & CPU_GHZ columns


def add_CPU_ID(df):
    df = df.merge(df_CPU, on=["CPU_Brand", "CPU_Model", "CPU_GHZ"], how="left")
    df.drop(columns=["CPU_Brand", "CPU_Model", "CPU_GHZ"], axis=1, inplace=True)

    return df

In [363]:
# Screen :: Create Schematic dataframe & CSV

Screen_cols = ["Screen_Size", "Screen_Type", "Screen_resolution"]

df_Screen = (
    df[Screen_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_Screen["index"] += 1

df_Screen.rename(columns={"index": "Screen_ID"}, inplace=True)

df_Screen.to_csv("../laplab_refer/csv_4_sql/Screen.csv", index=False)

df_Screen.head()

,Screen_ID,Screen_Size,Screen_Type,Screen_resolution
0,1,15.6,Full HD,1920x1080
1,2,13.3,IPS Panel Full HD / Touchscreen,1920x1080
2,3,15.6,IPS Panel Full HD,1920x1080
3,4,13.3,IPS Panel Full HD,1920x1080
4,5,14.0,Full HD,1920x1080


In [364]:
# Create function to add Screen_ID to the given dataframe and drop Screen_Size, Screen_Type & Screen_resolution columns


def add_Screen_ID(df):
    df = df.merge(df_Screen, on=["Screen_Size", "Screen_Type", "Screen_resolution"], how="left")
    df.drop(columns=["Screen_Size", "Screen_Type", "Screen_resolution"], axis=1, inplace=True)

    return df

In [365]:
# Storage :: Create Schematic dataframe & CSV

Storage_cols = [
    "Storage_SSD",
    "Storage_HDD",
    "Storage_Hybrid",
    "Storage_Flash",
    "Storage_Total",
]

df_Storage = (
    df[Storage_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_Storage["index"] += 1

df_Storage.rename(columns={"index": "Storage_ID"}, inplace=True)

df_Storage.to_csv("../laplab_refer/csv_4_sql/Storage.csv", index=False)

df_Storage.head()

,Storage_ID,Storage_SSD,Storage_HDD,Storage_Hybrid,Storage_Flash,Storage_Total
0,1,256,0,0,0,256
1,2,0,1024,0,0,1024
2,3,512,1024,0,0,1536
3,4,0,500,0,0,500
4,5,512,0,0,0,512


In [366]:
# Create function to add Storage_ID to the given dataframe and drop Storage_SSD, Storage_HDD, Storage_Hybrid, Storage_Flash & Storage_Total columns


def add_Storage_ID(df):
    df = df.merge(
        df_Storage,
        on=[
            "Storage_SSD",
            "Storage_HDD",
            "Storage_Hybrid",
            "Storage_Flash",
            "Storage_Total",
        ],
        how="left",
    )
    df.drop(
        columns=[
            "Storage_SSD",
            "Storage_HDD",
            "Storage_Hybrid",
            "Storage_Flash",
            "Storage_Total",
        ],
        axis=1,
        inplace=True,
    )

    return df

In [367]:
# GPU :: Create Schematic dataframe & CSV

GPU_cols = ["GPU_Brand", "GPU_Model"]

df_GPU = (
    df[GPU_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_GPU["index"] += 1

df_GPU.rename(columns={"index": "GPU_ID"}, inplace=True)

df_GPU.to_csv("../laplab_refer/csv_4_sql/GPU.csv", index=False)

df_GPU.head()

,GPU_ID,GPU_Brand,GPU_Model
0,1,AMD,Radeon R5 M420
1,2,Intel,HD Graphics 620
2,3,Nvidia,GeForce GTX 960M
3,4,AMD,Radeon 530
4,5,Nvidia,GeForce 940MX


In [368]:
# Create function to add GPU_ID to the given dataframe and drop GPU_Brand & GPU_Model columns


def add_GPU_ID(df):
    df = df.merge(df_GPU, on=["GPU_Brand", "GPU_Model"], how="left")
    df.drop(columns=["GPU_Brand", "GPU_Model"], axis=1, inplace=True)

    return df

In [369]:
# Laptop :: Create Schematic dataframe & CSV
    #   In two steps:
        #   1. Create Laptop dataframe with all columns needed
        #   2. Replace Foreign Key Columns with their respective IDs

Laptop_cols_original = [
    "Model_Name",
    "Category",
    "Weight",
    "RAM",
    "Price",
    "Discount",
    "Rating",
    "Store",
]
Laptop_cols_foreign = (
    Manufacturer_cols
    + OperatingSystem_cols
    + CPU_cols
    + Screen_cols
    + Storage_cols
    + GPU_cols
)

Laptop_cols_as_ID = [
    "Manufacturer_ID",
    "OperatingSystem_ID",
    "CPU_ID",
    "Screen_ID",
    "Storage_ID",
    "GPU_ID",
]

Laptop_cols_before_replace = Laptop_cols_original + Laptop_cols_foreign

df_Laptop = (
    df[Laptop_cols_before_replace]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_Laptop["index"] += 1

df_Laptop.rename(columns={"index": "Laptop_ID"}, inplace=True)

df_Laptop = df_Laptop.merge(df_Manufacturer, how="left", on=Manufacturer_cols).drop(
    columns=Manufacturer_cols, axis=1
)
df_Laptop = df_Laptop.merge(
    df_OperatingSystem, how="left", on=OperatingSystem_cols
).drop(columns=OperatingSystem_cols, axis=1)
df_Laptop = df_Laptop.merge(df_CPU, how="left", on=CPU_cols).drop(
    columns=CPU_cols, axis=1
)
df_Laptop = df_Laptop.merge(df_Screen, how="left", on=Screen_cols).drop(
    columns=Screen_cols, axis=1
)
df_Laptop = df_Laptop.merge(df_Storage, how="left", on=Storage_cols).drop(
    columns=Storage_cols, axis=1
)
df_Laptop = df_Laptop.merge(df_GPU, how="left", on=GPU_cols).drop(
    columns=GPU_cols, axis=1
)

df_Laptop.to_csv("../laplab_refer/csv_4_sql/Laptop.csv", index=False)

Laptop_cols = Laptop_cols_original + Laptop_cols_as_ID

df_Laptop.head()

,Laptop_ID,Model_Name,Category,Weight,RAM,Price,Discount,Rating,Store,Manufacturer_ID,OperatingSystem_ID,CPU_ID,Screen_ID,Storage_ID,GPU_ID
0,1,Vostro 3568,Notebook,2.18,8,3450000,0,NaN,NaN,1,1,1,1,1,1
1,2,Yoga 720-13IKB,2 in 1 Convertible,1.30,8,3400000,0,NaN,NaN,2,1,1,2,1,2
2,3,IdeaPad Y700-15ISK,Notebook,2.60,8,44900000,0,NaN,NaN,2,1,2,3,2,3
3,4,Inspiron 5370,Ultrabook,1.40,8,12550000,0,NaN,NaN,1,1,3,4,1,4
4,5,Aspire E5-576G,Notebook,2.23,4,2050000,0,NaN,NaN,3,1,1,1,1,5


In [370]:
# Create function to add Laptop_ID to the given dataframe and drop Laptop columns

def add_Laptop_ID(df):
    df = add_Manufacturer_ID(df)
    df = add_OperatingSystem_ID(df)
    df = add_CPU_ID(df)
    df = add_Screen_ID(df)
    df = add_Storage_ID(df)
    df = add_GPU_ID(df)

    df = df.merge(df_Laptop, on=Laptop_cols, how="left")
    df.drop(columns=Laptop_cols, axis=1, inplace=True)

    return df

In [371]:
# Orders :: Create Schematic dataframe & CSV

Orders_cols = ["Branch", "Order_Date", "Order_Priority"]

df_Orders = (
    df[Orders_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_Orders["index"] += 1

df_Orders.rename(columns={"index": "Orders_ID"}, inplace=True)

df_Orders.to_csv("../laplab_refer/csv_4_sql/Orders.csv", index=False)

df_Orders.head()

,Orders_ID,Branch,Order_Date,Order_Priority
0,1,Hamedan,1396-10-26,H
1,2,Kermanshah,1394-04-21,H
2,3,Rasht,1402-05-24,M
3,4,Kermanshah,1397-06-17,H
4,5,Karaj,1392-09-30,L


In [372]:
# Create function to add Orders_ID to the given dataframe and drop Branch, Order_Date & Order_Priority columns

def add_Orders_ID(df):
    df = df.merge(df_Orders, on=["Branch", "Order_Date", "Order_Priority"], how="left")
    df.drop(columns=["Branch", "Order_Date", "Order_Priority"], axis=1, inplace=True)

    return df

In [373]:
# OrderDetail :: Create Schematic dataframe & CSV
    # In four steps:
        # 1. Add Laptop ID to the dataframe & remove Laptop columns
        # 2. Add Orders ID to the dataframe & remove Orders columns
        # 3. Create OrderDetail dataframe with all columns needed
        # 4. Replace Foreign Key Columns with their respective IDs

OrderDetail_cols = ["Quantity", "Total_Price", "Profit", "Ship_Duration"] + [
    "Laptop_ID",
    "Orders_ID",
]

df_OrderDetail = add_Laptop_ID(df)
df_OrderDetail = add_Orders_ID(df_OrderDetail)

df_OrderDetail = (
    df_OrderDetail[OrderDetail_cols]
    .copy()
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
)
df_OrderDetail["index"] += 1

df_OrderDetail.rename(columns={"index": "OrderDetail_ID"}, inplace=True)

df_OrderDetail.to_csv("../laplab_refer/csv_4_sql/OrderDetail.csv", index=False)

df_OrderDetail

,OrderDetail_ID,Quantity,Total_Price,Profit,Ship_Duration,Laptop_ID,Orders_ID
0,1,1,3450000,250000,3,1,1
1,2,1,3400000,380000,2,2,2
2,3,1,44900000,5630000,5,3,3
3,4,1,12550000,1130000,3,4,4
4,5,1,2050000,180000,8,5,5
...,...,...,...,...,...,...,...
1017071,1017072,1,3800000,460000,4,26231,71452
1017072,1017073,1,1300000,120000,6,119385,46751
1017073,1017074,1,16700000,1700000,2,409689,103733
1017074,1017075,1,3750000,420000,10,34409,157279


In [374]:
# Create function to add OrderDetail_ID to the given dataframe and drop OrderDetail columns

def add_OrderDetail_ID(df):
    df = df.merge(df_OrderDetail, on=OrderDetail_cols, how="left")
    df.drop(columns=OrderDetail_cols, axis=1, inplace=True)

    return df